In [1]:
# import necessary libraries
import pandas as pd
import re
from airportsdata import load
from actions.done.CreateZFWMessageAction import CreateZFWMessageAction
from actions.done.UpdateEstimatesAction import UpdateEstimatesAction
from actions.done.CalculateWeightAndTrimAction import CalculateWeightAndTrimAction
from actions.done.CreateLoadsheetAction import CreateLoadsheetAction
import json
import os
import concurrent.futures
import time

def load_config(path):
    with open(path, 'r') as file:
        return json.load(file)

config_path = '../pandas_config.json'
config = load_config(config_path)
parquet_config = config['parquet']

# Setting up the engine as a global constant
ENGINE = parquet_config['engine']
COMPRESSION = parquet_config['compression']
FILE_PATH = "../data"

In [2]:
df = pd.read_parquet(f"{FILE_PATH}/data_parquet/processed_data_combined.parquet", engine=ENGINE)

In [32]:
KEEP_COLUMNS = ["flight_id", "action_name","entry_details"]
df = df[KEEP_COLUMNS]

In [34]:
os.listdir(f"actions/done")
ACTIONS = [action.replace(".py", "") for action in os.listdir(f"actions/done") if action.endswith(".py")]
df = df[df['action_name'].isin(ACTIONS)]

In [35]:
flight_ids = df['flight_id'].unique()
# Set flight_id as index for faster access
df.set_index('flight_id', inplace=True)

In [38]:
def process_flight(flight, df):
    subset_df = df.loc[flight]
    subset_df.sort_index(inplace=True)
    actions = subset_df['action_name'].values
    entry_details = subset_df["entry_details"].values
    temp_estimated_zfws = []
    temp_actual_zfws = []
    temp_actual_fuels = []
    temp_actual_tows = []
    
    for entry, action in zip(entry_details, actions):
        if action == "CreateZFWMessageAction":
            estimated_zfw, actual_zfw = CreateZFWMessageAction(entry)
            if estimated_zfw is not None:
                temp_estimated_zfws.append(estimated_zfw)
            if actual_zfw is not None:
                temp_actual_zfws.append(actual_zfw)
        elif action == "UpdateEstimatesAction":
            estimated_zfw = UpdateEstimatesAction(entry)
            if estimated_zfw is not None:
                temp_estimated_zfws.append(estimated_zfw)
            if actual_zfw is not None:
                temp_actual_zfws.append(actual_zfw)
        elif action == "CalculateWeightAndTrimAction":
            actual_zfw = CalculateWeightAndTrimAction(entry)
            if actual_zfw is not None:
                temp_actual_zfws.append(actual_zfw)
        elif action == "CreateLoadsheetAction":
            actual_tow, actual_zfw, estimated_zfw, actual_fuel = CreateLoadsheetAction(entry)
            if actual_zfw is not None:
                temp_actual_zfws.append(actual_zfw)
            if estimated_zfw is not None:
                temp_estimated_zfws.append(estimated_zfw)
            if actual_fuel is not None:
                temp_actual_fuels.append(actual_fuel)
            if actual_tow is not None:
                temp_actual_tows.append(actual_tow)
    
    return flight, temp_estimated_zfws, temp_actual_zfws, temp_actual_fuels, temp_actual_tows

def collect_results(result):
    flight, temp_estimated_zfws, temp_actual_zfws, temp_actual_fuels, temp_actual_tows = result
    estimated_zfws[flight] = temp_estimated_zfws
    actual_zfws[flight] = temp_actual_zfws
    actual_take_off_fuels[flight] = temp_actual_fuels
    actual_tows[flight] = temp_actual_tows

start = time.time()
estimated_zfws = {}
actual_zfws = {}
actual_take_off_fuels = {}
actual_tows = {}

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(process_flight, flight, df) for flight in flight_ids[:300]]
    for future in concurrent.futures.as_completed(futures):
        collect_results(future.result())

end = time.time()
print(end - start)


2.1750781536102295


In [14]:
len(flight_ids)

14447

In [140]:
estimated_zfws_tuples = [(k, v) for k, v in estimated_zfws.items()]
actual_zfws_tuples = [(k, v) for k, v in actual_zfws.items()]

# Create DataFrame
estimated_zfws_df = pd.DataFrame(estimated_zfws_tuples, columns=['flight_number', 'estimated_zfws'])
actual_zfws_df = pd.DataFrame(actual_zfws_tuples, columns=['flight_number', 'actual_zfws'])
weights = pd.merge(estimated_zfws_df, actual_zfws_df, on='flight_number', how="outer")


In [7]:
weights.to_parquet("../data/data_parquet/weights.parquet", engine=ENGINE, compression=COMPRESSION)